In [11]:
"""
🎬 BEHIND THE SCENES: How Language Models Generate Text
A visual, step-by-step demonstration of what happens inside an LLM

Perfect for non-technical audiences!
"""

import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time


# SETUP: Load the Model
print("\n" + "🎬" * 35)
print("   WELCOME TO: INSIDE THE AI BRAIN")
print("🎬" * 35 + "\n")

print("📦 Loading the AI model (Phi-3)...")
print("   Think of this as loading a very smart calculator that understands language\n")

MODEL_PATH = "/Users/somesh/How Transformer LLMs Work/Transformer Architecture/models/microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=True,
)

print("Model loaded! Let's see what's inside...\n")

# PART 1: MODEL ANATOMY
print("=" * 80)
print("🔍 PART 1: WHAT'S INSIDE THE MODEL?")
print("=" * 80)

print("\n📊 MODEL STATISTICS:")
print("-" * 80)
print(f"  • Vocabulary Size:    {model.config.vocab_size:,} words/tokens")
print(f"  • Embedding Dimension: {model.config.hidden_size:,} (how big each word's 'meaning vector' is)")
print(f"  • Number of Layers:    {model.config.num_hidden_layers} (like floors in a building)")
print(f"  • Attention Heads:     {model.config.num_attention_heads} (parallel thinking pathways)")
print(f"  • Parameters:          ~3.8 billion tiny numbers that store knowledge")
print("\n💡 Analogy: Imagine a library with 32,064 books, stacked 32 floors high,")
print("     with 32 librarians on each floor helping you find information!\n")

# Show the architecture
print("\n🏗️  MODEL ARCHITECTURE (The Blueprint):")
print("-" * 80)
print("""
┌─────────────────────────────────────────────────────┐
│  INPUT: "The capital of France is"                  │
└─────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────┐
│  STEP 1: Token Embeddings                           │
│  Converts words → numbers → meaning vectors         │
│  Size: 32,064 vocabulary × 3,072 dimensions         │
└─────────────────────────────────────────────────────┘
                        ↓
┌────────────────────────────────────────────────────┐
│  STEP 2: 32 Transformer Layers (The Brain)         │
│  ┌───────────────────────────────────────────┐     │
│  │ Layer 1:  Self-Attention + Neural Network │     │
│  │ Layer 2:  Self-Attention + Neural Network │     │
│  │ Layer 3:  Self-Attention + Neural Network │     │
│  │    ...    (each layer learns patterns)    │     │
│  │ Layer 32: Self-Attention + Neural Network │     │
│  └───────────────────────────────────────────┘     │
└────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────┐
│  STEP 3: Language Model Head                        │
│  Predicts next word from 32,064 possibilities       │
└─────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────┐
│  OUTPUT: "Paris" (with 87.3% confidence!)           │
└─────────────────────────────────────────────────────┘
""")

input("\n⏸️  Press ENTER to continue to the live demonstration...")

# PART 2: LIVE DEMONSTRATION
print("\n" + "=" * 80)
print("🎥 PART 2: LIVE GENERATION - BEHIND THE SCENES")
print("=" * 80)

prompt = "The capital of France is"
print(f"\n📝 INPUT PROMPT: \"{prompt}\"")
print("-" * 80)


# Step 1: Tokenization
print("\n🔤 STEP 1: TOKENIZATION (Breaking text into pieces)")
print("-" * 80)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
tokens = [tokenizer.decode([id]) for id in input_ids[0]]

print("\n  Original text: \"The capital of France is\"")
print("           ↓")
print("  Broken into tokens:")
for i, (token, token_id) in enumerate(zip(tokens, input_ids[0])):
    print(f"    Token {i+1}: '{token}' → ID: {token_id.item()}")

print(f"\n    Total input tokens: {len(tokens)}")
print("\n  💡 Why tokenization? Computers only understand numbers!")
print("       Each word/piece gets a unique ID from 0 to 32,063")

input("\n⏸️  Press ENTER to see embeddings...")

# Step 2: Embeddings
print("\n📊 STEP 2: EMBEDDINGS (Converting IDs to meaning vectors)")
print("-" * 80)

with torch.no_grad():
    embeddings = model.model.embed_tokens(input_ids)

print(f"\n  Shape: {embeddings.shape}")
print(f"         ↓")
print(f"  Batch size: {embeddings.shape[0]} (how many sentences)")
print(f"  Tokens:     {embeddings.shape[1]} (words in sentence)")
print(f"  Dimensions: {embeddings.shape[2]} (size of each meaning vector)")

print(f"\n  Example: First token '{tokens[0]}' becomes a vector of 3,072 numbers:")
print(f"  [{embeddings[0, 0, 0]:.4f}, {embeddings[0, 0, 1]:.4f}, {embeddings[0, 0, 2]:.4f}, ... {embeddings[0, 0, -1]:.4f}]")

print("\n 💡Think of each vector as coordinates in a 3,072-dimensional space!")
print("     Similar words (like 'king' and 'queen') are close together")

input("\n⏸️ Press ENTER to see the model thinking...")

# Step 3: Through the layers
print("\n🧠 STEP 3: PROCESSING THROUGH 32 LAYERS (The AI is thinking!)")
print("-" * 80)

print("\n  Each layer does two things:")
print("    1️⃣  ATTENTION: 'Which words should I focus on?'")
print("    2️⃣  PROCESSING: 'What does this mean?'")
print("\n  Let's watch as information flows through the layers...\n")

layer_outputs = []
hidden_states = embeddings.clone()

with torch.no_grad():
    for i, layer in enumerate(model.model.layers):
        seq_len = hidden_states.shape[1]
        position_ids = torch.arange(
            seq_len,
            device=hidden_states.device
        ).unsqueeze(0)

        if i < 3 or i == 15 or i == 31:
            print(f"  🔄 Layer {i+1}/32 processing...")
        elif i == 3:
            print("  ⏩ Layers 4–31 processing...")

        layer_output = layer(
            hidden_states,
            attention_mask=None,
            position_ids=position_ids,
            use_cache=False,
            output_attentions=False,
        )

        hidden_states = layer_output[0]

        if i in [0, 15, 31]:
            layer_outputs.append(hidden_states.clone())

print("\n ✅All 32 layers complete!")
print("\n 💡Each layer refines the understanding:")
print("     Early layers (1-10):   Learn grammar and syntax")
print("     Middle layers (11-20): Learn meaning and relationships")
print("     Late layers (21-32):   Learn specific knowledge (like 'capital of France = Paris')")

input("\n⏸️ Press ENTER to see the final prediction...")

# Step 4: Generate next token
print("\n🎯 STEP 4: PREDICTING THE NEXT WORD")
print("-" * 80)

with torch.no_grad():
    # Get final output
    outputs = model(input_ids, use_cache=False)
    logits = outputs.logits[0, -1, :]  # Last token's predictions

    # Get probabilities
    probs = torch.softmax(logits, dim=-1)

    # Get top 10 predictions
    top_k = 10
    top_probs, top_indices = torch.topk(probs, top_k)

print("\n  The model's brain has 32,064 neurons, each voting for a different word.")
print("  Let's see the TOP 10 candidates:\n")
print("-" * 80)
print(f"  {'Rank':<6}{'Word':<20}{'Confidence':<15}{'Probability':<15}")
print("-" * 80)

for rank, (idx, prob) in enumerate(zip(top_indices, top_probs), 1):
    token = tokenizer.decode(idx)
    bar_length = int(prob.item() * 50)
    bar = "█" * bar_length
    print(f"  {rank:<6}{repr(token):<20}{prob.item()*100:>6.2f}%{'':>8}{bar}")

winner = tokenizer.decode(top_indices[0])
winner_prob = top_probs[0].item() * 100

print("-" * 80)
print(f"\n  🏆 WINNER: '{winner}' with {winner_prob:.2f}% confidence!")
print(f"\n  Full answer: \"{prompt}{winner}\"")

input("\n⏸️  Press ENTER to generate a full sentence...")

# PART 3: FULL GENERATION
print("\n" + "=" * 80)
print("🎬 PART 3: GENERATING MULTIPLE WORDS (AUTOREGRESSIVE GENERATION)")
print("=" * 80)

prompt2 = "The capital of France is"
max_tokens = 20

print(f"\n📝 Starting prompt: \"{prompt2}\"")
print("\n  The model will generate one word at a time, adding each new word")
print("  to the prompt and predicting the next word. Let's watch!\n")
print("-" * 80)

input_ids = tokenizer(prompt2, return_tensors="pt").input_ids
generated_ids = input_ids.clone()
generated_text = prompt2

print(f"  Starting: \"{generated_text}\"")
print()

with torch.no_grad():
    for step in range(max_tokens):
        # Generate next token
        outputs = model(generated_ids, use_cache=False)
        next_token_id = outputs.logits[0, -1, :].argmax(-1)
        next_token = tokenizer.decode(next_token_id)

        # Get top 3 for this step
        probs = torch.softmax(outputs.logits[0, -1, :], dim=-1)
        top3_probs, top3_indices = torch.topk(probs, 3)

        # Show the thinking
        print(f"  Step {step+1}:")
        print(f"    Current text: \"{generated_text}\"")
        print(f"    Top 3 next words:")
        for i, (idx, prob) in enumerate(zip(top3_indices, top3_probs), 1):
            token = tokenizer.decode(idx)
            marker = "✓" if i == 1 else " "
            print(f"      {marker} {i}. '{token}' ({prob.item()*100:.1f}%)")
        print(f"    → Chose: '{next_token}'")

        # Update
        generated_text += next_token
        generated_ids = torch.cat([generated_ids, next_token_id.unsqueeze(0).unsqueeze(0)], dim=1)

        print(f"    New text: \"{generated_text}\"")
        print()

        # Stop at end of sentence or period
        if next_token_id.item() == tokenizer.eos_token_id or "." in next_token:
            print(f"✅ Stopped at step {step+1} (found ending)")
            break

        time.sleep(0.5)  # Pause for effect

print("-" * 80)
print(f"\n  🎉 FINAL RESULT: \"{generated_text}\"")

# PART 4: KEY INSIGHTS
print("\n" + "=" * 80)
print("💡 KEY INSIGHTS: WHAT DID WE LEARN?")
print("=" * 80)

print("""
1. 🔤 TOKENIZATION
   Text is broken into pieces (tokens) and converted to numbers.
   The model has a vocabulary of 32,064 different tokens.

2. 📊 EMBEDDINGS
   Each token becomes a vector of 3,072 numbers that captures its meaning.
   Similar words have similar vectors.

3. 🧠 32 TRANSFORMER LAYERS
   Each layer has:
   • ATTENTION: Looks at relationships between words
   • MLP: Processes and transforms information

   Early layers learn grammar, late layers learn facts!

4. 🎯 PREDICTION
   The final layer outputs scores for all 32,064 possible next words.
   We pick the highest score (or sample based on probabilities).

5. 🔁 AUTOREGRESSIVE
   To generate multiple words, we:
   → Predict one word
   → Add it to the input
   → Predict the next word
   → Repeat!

6. 📈 CONFIDENCE
   The model assigns probability to each possible next word.
   Higher probability = more confident!

7. 🎲 RANDOMNESS (Optional)
   • Greedy: Always pick highest probability (deterministic)
   • Sampling: Pick randomly based on probabilities (creative)
   • Temperature: Control randomness (higher = more random)
""")

print("=" * 80)
print("\n✨ THE MAGIC: The model learned all this from reading massive amounts of text!")
print("   It discovered patterns about language, grammar, facts, and reasoning")
print("   just by trying to predict the next word billions of times!\n")
print("=" * 80)


# INTERACTIVE DEMO
print("\n" + "🎮" * 35)
print("YOUR TURN: TRY YOUR OWN PROMPT!")
print("🎮" * 35 + "\n")

while True:
    user_prompt = input("\n✏️ Enter your prompt (or 'quit' to exit): ").strip()

    if user_prompt.lower() in ['quit', 'exit', 'q']:
        print("\n👋 Thanks for exploring! Now you understand how LLMs work!\n")
        break

    if not user_prompt:
        continue

    print(f"\n🎬 Generating response for: \"{user_prompt}\"")
    print("-" * 80)

    input_ids = tokenizer(user_prompt, return_tensors="pt").input_ids
    generated_ids = input_ids.clone()
    generated_text = ""

    with torch.no_grad():
        for i in range(50):
            outputs = model(generated_ids, use_cache=False)
            next_token_id = outputs.logits[0, -1, :].argmax(-1)
            token = tokenizer.decode(next_token_id)
            generated_text += token

            print(token, end='', flush=True)

            generated_ids = torch.cat([
                generated_ids,
                next_token_id.unsqueeze(0).unsqueeze(0)
            ], dim=1)

            if next_token_id.item() == tokenizer.eos_token_id:
                break

            time.sleep(0.05)

    print(f"\n\n✅ Complete response: {user_prompt}{generated_text}")
    print("-" * 80)


🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬
   WELCOME TO: INSIDE THE AI BRAIN
🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬🎬

📦 Loading the AI model (Phi-3)...
   Think of this as loading a very smart calculator that understands language



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 209.23it/s]

Model loaded! Let's see what's inside...

🔍 PART 1: WHAT'S INSIDE THE MODEL?

📊 MODEL STATISTICS:
--------------------------------------------------------------------------------
  • Vocabulary Size:    32,064 words/tokens
  • Embedding Dimension: 3,072 (how big each word's 'meaning vector' is)
  • Number of Layers:    32 (like floors in a building)
  • Attention Heads:     32 (parallel thinking pathways)
  • Parameters:          ~3.8 billion tiny numbers that store knowledge

💡 Analogy: Imagine a library with 32,064 books, stacked 32 floors high,
     with 32 librarians on each floor helping you find information!


🏗️  MODEL ARCHITECTURE (The Blueprint):
--------------------------------------------------------------------------------

┌─────────────────────────────────────────────────────┐
│  INPUT: "The capital of France is"                  │
└─────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────┐
│  S


🎥 PART 2: LIVE GENERATION - BEHIND THE SCENES

📝 INPUT PROMPT: "The capital of France is"
--------------------------------------------------------------------------------

🔤 STEP 1: TOKENIZATION (Breaking text into pieces)
--------------------------------------------------------------------------------

  Original text: "The capital of France is"
           ↓
  Broken into tokens:
    Token 1: 'The' → ID: 450
    Token 2: 'capital' → ID: 7483
    Token 3: 'of' → ID: 310
    Token 4: 'France' → ID: 3444
    Token 5: 'is' → ID: 338

    Total input tokens: 5

  💡 Why tokenization? Computers only understand numbers!
       Each word/piece gets a unique ID from 0 to 32,063

📊 STEP 2: EMBEDDINGS (Converting IDs to meaning vectors)
--------------------------------------------------------------------------------

  Shape: torch.Size([1, 5, 3072])
         ↓
  Batch size: 1 (how many sentences)
  Tokens:     5 (words in sentence)
  Dimensions: 3072 (size of each meaning vector)

  Example: Fi